In [1]:
#!pip install tensorflow==2.15.0 tensorflow-hub keras==2.15.0
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [2]:
train_data, validation_data, test_data = tfds.load(name="imdb_reviews",
                                                   split=('train[:60%]', 'train[60%:]', 'test'),
                                                   as_supervised=True)

In [3]:
train_data
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

GPU is NOT AVAILABLE


In [4]:
train_example_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [5]:
train_example_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [6]:
 train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [7]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

In [8]:
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)

In [9]:
hub_layer(train_example_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [10]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400373 (1.53 MB)
Trainable params: 400373 (1.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [13]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 5s 93ms/step - loss: 0.7721 - accuracy: 0.4987 - val_loss: 0.6969 - val_accuracy: 0.5375
Epoch 2/20
30/30 [==============================] - 2s 69ms/step - loss: 0.6611 - accuracy: 0.5652 - val_loss: 0.6320 - val_accuracy: 0.6039
Epoch 3/20
30/30 [==============================] - 2s 68ms/step - loss: 0.6112 - accuracy: 0.6243 - val_loss: 0.5977 - val_accuracy: 0.6427
Epoch 4/20
30/30 [==============================] - 3s 108ms/step - loss: 0.5762 - accuracy: 0.6685 - val_loss: 0.5650 - val_accuracy: 0.6867
Epoch 5/20
30/30 [==============================] - 2s 69ms/step - loss: 0.5378 - accuracy: 0.7065 - val_loss: 0.5267 - val_accuracy: 0.7209
Epoch 6/20
30/30 [==============================] - 2s 69ms/step - loss: 0.4959 - accuracy: 0.7396 - val_loss: 0.4931 - val_accuracy: 0.7585
Epoch 7/20
30/30 [==============================] - 2s 67ms/step - loss: 0.4560 - accuracy: 0.7728 - val_loss: 0.4617 - val_accuracy: 0.7821
Epoch 8/20
3

In [20]:
history = model.fit(train_data.shuffle(10000).batch(500),
                    epochs=25,
                    validation_data=validation_data.batch(500),
                    verbose=1)

Epoch 1/25
30/30 [==============================] - 3s 102ms/step - loss: 1.0630e-04 - accuracy: 1.0000 - val_loss: 1.1546 - val_accuracy: 0.8516
Epoch 2/25
30/30 [==============================] - 2s 71ms/step - loss: 1.0388e-04 - accuracy: 1.0000 - val_loss: 1.1576 - val_accuracy: 0.8518
Epoch 3/25
30/30 [==============================] - 2s 71ms/step - loss: 1.0190e-04 - accuracy: 1.0000 - val_loss: 1.1605 - val_accuracy: 0.8516
Epoch 4/25
30/30 [==============================] - 2s 70ms/step - loss: 1.0006e-04 - accuracy: 1.0000 - val_loss: 1.1629 - val_accuracy: 0.8518
Epoch 5/25
30/30 [==============================] - 3s 104ms/step - loss: 9.7792e-05 - accuracy: 1.0000 - val_loss: 1.1650 - val_accuracy: 0.8516
Epoch 6/25
30/30 [==============================] - 2s 69ms/step - loss: 9.5964e-05 - accuracy: 1.0000 - val_loss: 1.1682 - val_accuracy: 0.8519
Epoch 7/25
30/30 [==============================] - 2s 69ms/step - loss: 9.4189e-05 - accuracy: 1.0000 - val_loss: 1.1705 - val_

In [21]:
results = model.evaluate(test_data.batch(500), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

50/50 - 2s - loss: 1.3872 - accuracy: 0.8312 - 2s/epoch - 34ms/step
loss: 1.387
accuracy: 0.831
